The Australian DEM is 35GB, and hosted by the DEA [here](https://data.dea.ga.gov.au/?prefix=projects/elevation/ga_srtm_dem1sv1_0/)
AWS bucket name to use to access the data is `dea-public-data`

## Original Data Source

*SRTM-derived 1 Second Digital Elevation Models Version 1.0*

The 1 second Shuttle Radar Topography Mission (SRTM) Digital Elevation
Models Version 1.0 package comprises three surface models: the Digital
Elevation Model (DEM), the Smoothed Digital Elevation Model (DEM-S)
and the Hydrologically Enforced Digital Elevation Model (DEM-H).

### sources and guides
[DEA guide to accessing data from their public S3 bucket using STAC](https://docs.dea.ga.gov.au/guides/setup/gis/stac/)

[Downloading and streaming data using STAC metadata](https://docs.dea.ga.gov.au/notebooks/How_to_guides/Downloading_data_with_STAC/)

In [1]:
import os

# packages for widgets and interactive map:
import ipywidgets as widgets
from IPython.display import display, HTML
from ipyleaflet import Map, WidgetControl, GeoData
import io
import base64
import rioxarray
import xarray as xr
import xarray_leaflet
import matplotlib.pyplot as plt

from localtileserver import get_leaflet_tile_layer

#packages for geo-processing:
import geopandas as gpd
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.windows import from_bounds
import xarray as xr
import xarray_leaflet

import pystac_client
import stackstac
import odc.stac
import pyproj
from dask.distributed import Client, LocalCluster
import pandas as pd

1. Set environment variable for AWS and start a Dask client for parallel computation (needed for stackstac)
2. Read the boundary GeoJSON file and extract the bounding box
3. Open the STAC catalogs for DEM and RGB data
4. Define the collections for DEM and RGB data
5. Set the date range for the RGB data
6. Define spatial projection information

## environment and STAC setup

In [2]:
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'
client = Client()
#cluster = LocalCluster()

"""
If STAC service is down, try using direct link to the DEM-H geotiff file
"""
dem_href = 'https://dea-public-data.s3-ap-southeast-2.amazonaws.com/projects/elevation/ga_srtm_dem1sv1_0/dem1sv1_0.tif'


#catalog_dem = pystac_client.Client.open("https://explorer.sandbox.dea.ga.gov.au/stac/")
#collection_dem = ['ga_srtm_dem1sv1_0']

crs_string = "epsg:3857"
epsg = pyproj.CRS.from_string(crs_string).to_epsg()
resolution = 90 #added to resolve error "Failed to auto-guess CRS/resolution."


## interactive map so users can upload aoi files

In [3]:
style = {'description_width': 'initial'}
uploader = widgets.FileUpload(
    description='Upload file',
    accept='.zip, .json, .geojson',
    multiple=False,
    button_style='primary',
    style=style,
)

submit = widgets.Button(description='Display AOI', button_style='success', tooltip='Click me', style=style)

reset = widgets.Button(description='Reset', button_style='warning', tooltip='Click me', style=style)

get_data = widgets.Button(description='Get data for AOI', button_style='primary', tooltip='Click me', style=style)


the center point and zoom of the map are currently hard coded. Need to be updated to "zoom to layer" so that the map centers on the area covered by the user-uploaded geojson.
Also note that the map center lat/long are in opposite order to how they are stored in a geojson so need to add line to swap x/y coords to be used.

In [4]:
Map = Map(height='500px', width='500px', center=(-29.2253, 116.26012), zoom=10) #note that the center coordinates are the opposite to what is stored in a geojson, you need to swap the lat/long

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

with output_widget:
    display(uploader)
    display(submit)
    display(get_data)
    display(reset)

In [5]:
geo_json = None

def get_vector(upload_widget):
    vector = None
    try:
        file = upload_widget.value[0]
        content = file['content']
        bytes_io = io.BytesIO(content)
        vector = gpd.read_file(bytes_io)

    except Exception as e:
        print(e)
    return vector


def submit_clicked(b):
    global geo_json
    try:
        fc = get_vector(uploader)
        geo_json = GeoData(geo_dataframe = fc)
        Map.add_layer(geo_json)
    except Exception as e:
        print(e)
    
submit.on_click(submit_clicked)


def reset_clicked(b):
    global geo_json  # Declare geo_json as global so that we can access it
    output_widget.clear_output()
    with output_widget:
        display(uploader)
        display(submit)
        display(get_data)
        display(reset)
    uploader.value[0].clear()
    uploader._counter = 0
    if geo_json is not None:  # If a GeoData layer has been added to the map
        Map.remove_layer(geo_json)  # Remove the GeoData layer from the map
        geo_json = None  # Reset the reference to the GeoData layer

reset.on_click(reset_clicked)


## using uploaded aoi to get DEM

Alternative fetch DEM using direct href instead of doing a stac search because the DEA stac endpoint is having issues:

In [6]:
import tempfile
test_dem = None


def create_download_link(output_filename, title="Download GeoTIFF", filename="data.tif"):
    data = open(output_filename, "rb").read()
    b64 = base64.b64encode(data).decode()

    html = f'<a download="{filename}" href="data:application/octet-stream;base64,{b64}" target="_blank">{title}</a>'
    return HTML(html)

def get_data_clicked(b):
    global test_dem
    fc = get_vector(uploader) #don't use the geoData global variable, use the geodataframe
    bbox = list(fc.total_bounds)
     
    with rasterio.open(dem_href) as src:
        window=from_bounds(*bbox, transform=src.transform)
        data = src.read(window=window)
        kwargs = src.meta.copy()
        kwargs.update({
        'height': window.height,
        'width': window.width,
        'transform': rasterio.windows.transform(window, src.transform)})
        
    
        with rasterio.open('test-dem.tif', 'w', **kwargs) as dst:
            dst.write(data)
    
    test_dem = rioxarray.open_rasterio('test-dem.tif')
    #test_dem = test_dem.rio.reproject('EPSG:4326')
    test_dem = test_dem.sel(band=1)
    
    l = test_dem.leaflet.plot(Map)
    #Map.add_layer(test_dem)
    


get_data.on_click(get_data_clicked)
    



Download the fetched DEM raster:

In [7]:
# import tempfile
# test_dem = None
# # FILEPATH: /workspaces/data-notebooks/notebooks/evergreen-resources/get-elevation-data-for-aoi.ipynb
# #this cell is for the get_data button

# def create_download_link(output_filename, title="Download GeoTIFF", filename="data.tif"):
#     data = open(output_filename, "rb").read()
#     b64 = base64.b64encode(data).decode()

#     html = f'<a download="{filename}" href="data:application/octet-stream;base64,{b64}" target="_blank">{title}</a>'
#     return HTML(html)

# def get_data(b):
#     global test_dem
#     fc = get_vector(uploader) #don't use the geoData global variable, use the geodataframe
#     bbox = list(fc.total_bounds)
#     ds_dem = catalog_dem.search(
#     bbox=bbox, 
#     collections=collection_dem)

#     items = ds_dem.item_collection()
#     bands_of_interest = 'dem_h'

#     da = stackstac.stack(
#         items,
#         assets=[bands_of_interest],
#         epsg=epsg,
#         resolution=resolution,
#         bounds_latlon=bbox# <-- use Dask
#     ).isel(time=0)
#     da = da.compute()
#     test_dem = da
    
#     #da.leaflet.plot(Map, colormap=plt.cm.terrain)
    
#     # output_filename = "clipped_raster.tif"
#     # with rasterio.open(
#     #     output_filename,
#     #     'w',
#     #     driver='GTiff',
#     #     height=da.shape[1],
#     #     width=da.shape[2],
#     #     count=da.shape[0],
#     #     dtype=da.dtype,
#     #     crs=da.crs,
#     #     transform=da.transform,
#     # ) as dst:
#     #     dst.write(da)
        
#     # temp_raster = rioxarray.open_rasterio(output_filename)
#     # temp_raster_4326 = temp_raster.rio.reproject('EPSG:4326')
#     # temp_raster_4326_mapband = temp_raster_4326.sel(band=1)

#     # temp_raster_4326_mapband.leaflet.plot(Map, colormap=plt.cm.terrain)

#     #download_button = create_download_link(output_filename, title="Download GeoTIFF", filename="demdata.tif")

#     return da
    



## This section of code uses a hardcoded geojson to test. Remove once widget version is working.

In [8]:
# geom_gpd = gpd.read_file('data/temp/dissolved-boundaries.geojson')
# bbox = list(geom_gpd.total_bounds)
# bbox

Search for DEM data in the STAC catalog. Note that there is no timeframe filtering here because there is only 1, perpetual DEM in this catalog. If you add a timeframe you will get 0 results.
There are 3 assets in this collection: `dem`, `dem_s` and `dem_h`. All three are based on the NASA SRTM elevation data, so just use `dem_h`.
We will also get the url for the `dem_h` COG, so we can display it on a leafmap later on.

In [9]:
# ds_dem = catalog_dem.search(
#     bbox=bbox, 
#     collections=collection_dem)

# items = ds_dem.item_collection()
# print(f"Found {len(items)} items in the {collection_dem}")

In [10]:
# first_item = items.items[0]
# all_bands = list(first_item.assets.keys())
# print("Assets available:")
# print(*all_bands, sep=', ')

In [11]:
# bands_of_interest = 'dem_h'

# da = stackstac.stack(
#     items,
#     assets=[bands_of_interest],
#     epsg=epsg,
#     resolution=resolution,
#     bounds_latlon=bbox# <-- use Dask
# ).isel(time=0)
# da

Stack the DEM data, specifying the spatial bounds and desired resolution

One thing to watch out for with stackstac.stack is that you will wind up with a distinct time coordinate for each STAC item that you pass in. To achieve the intuitive representation of the data, you need to flatten the DataArray with respect to day.
Note: if you are only reading a single STAC item, stackstac.mosaic will inadvertently reduce your data along the band dimension (which is definitely not what you want!), hence the conditional statement checking for more than one time coordinate value.
[from here](https://hrodmn.dev/posts/stackstac/)

Here we are getting around the problem by using `isel` to select the first time epoch, as there is only one. A different approach will be needed if working with data across timeframes.

In [12]:
#da = da.compute()
# da.plot()

Save the selected DEM data as a GeoTIFF file

In [13]:
# with rasterio.open(dem_href) as dem_data:
#         # Read a window of data from the band
#         data = dem_data.read(window=from_bounds(*bbox, transform=dem_data.transform))
#         data = dem_data.read()



In [14]:

# output_filename = "data/temp/clipped_raster.tif"
# with rasterio.open(
#     output_filename,
#     'w',
#     driver='GTiff',
#     height=test_dem.shape[0],
#     width=test_dem.shape[1],
#     count=test_dem.shape[0],
#     dtype=test_dem.dtype,
#     crs=test_dem.crs,
#     transform=test_dem.transform,
# ) as dst:
#     dst.write(test_dem)

In [15]:
# # Define the filename for the reprojected raster
# reprojected_filename = "data/temp/reprojected_raster.tif"

# # Open the original clipped raster
# with rasterio.open(output_filename) as src:
#     # Define the transform and dimensions for the new raster
#     transform, width, height = calculate_default_transform(
#         src.crs, 'EPSG:4326', src.width, src.height, *src.bounds)
    
#     # Define the metadata for the new raster
#     kwargs = src.meta.copy()
#     kwargs.update({
#         'crs': 'EPSG:4326',
#         'transform': transform,
#         'width': width,
#         'height': height
#     })

#     # Create and write the reprojected raster
#     with rasterio.open(reprojected_filename, 'w', **kwargs) as dst:
#         for i in range(1, src.count + 1):
#             reproject(
#                 source=rasterio.band(src, i),
#                 destination=rasterio.band(dst, i),
#                 src_transform=src.transform,
#                 src_crs=src.crs,
#                 dst_transform=transform,
#                 dst_crs='EPSG:4326',
#                 resampling=Resampling.nearest)

In [16]:
# dem_client = TileClient('data/temp/reprojected_raster.tif')
# dem_client.thumbnail(palette="gist_earth")

## put map cell at bottom of notebook

In [17]:
Map

Map(center=[-29.2253, 116.26012], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [18]:
#display(download_button)

In [21]:
print(test_dem)

<xarray.DataArray (y: 313, x: 458)>
[143354 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 116.3 116.3 116.3 116.3 ... 116.4 116.4 116.4 116.4
  * y            (y) float64 -29.22 -29.22 -29.22 ... -29.31 -29.31 -29.31
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     -3.4028235e+38
    scale_factor:   1.0
    add_offset:     0.0


In [20]:
#plt.imshow(test_dem, cmap="gist_earth")